In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as soup
from urllib.request import Request,urlopen
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, os, csv
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [11]:
df = pd.read_csv(r'grxlup.csv')
lk = df['url']
lk[0:255]

0      https://www.goodrx.com/differin-gel?label_over...
1      https://www.goodrx.com/differin?label_override...
2      https://www.goodrx.com/adderall?label_override...
3      https://www.goodrx.com/adderall-xr?label_overr...
4      https://www.goodrx.com/concerta?label_override...
                             ...                        
250        https://www.goodrx.com/azo?label_override=azo
251    https://www.goodrx.com/bactrim?label_override=...
252    https://www.goodrx.com/macrobid?label_override...
253    https://www.goodrx.com/pyridium?label_override...
254    https://www.goodrx.com/marinol?label_override=...
Name: url, Length: 255, dtype: object

In [27]:
details_file = open('details_3.csv', 'w', newline="")
details_field_name = ['drug_id','link1', 'link2', 'brand_name','generic_name', 'description', 'form', "dosage", 'quantity', 'common_brands', 'drug_class', 'controlled_substance_classification', 'generic_status', 'availability']
details_writer = csv.DictWriter(details_file, fieldnames=details_field_name)
details_writer.writeheader()

167

In [28]:
drung_info_keys = ['availability', 'generic_status', 'controlled_substance_classification', 'drug_class', 'common_brands']
details_keys = ['description', 'generic_name', 'brand_name', 'form', 'quantity', 'dosage']

In [29]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-blink-features")
options.add_argument("--disable-blink-features=AutomationControlled")

In [30]:
driver = webdriver.Chrome(options=options, executable_path=r'D:\Software\chromedriver.exe')

C:\Users\astro\AppData\Local\Temp/ipykernel_9288/146559550.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=r'D:\Software\chromedriver.exe')


In [ ]:
count = 1
def get_drung_info(link):
    # declaring all required lists
    common_brands = []
    drug_class = []
    controlled_substance_classification = []
    generic_status = []
    availability = []

    # driver = webdriver.Chrome(r'C:\Users\conta\chromedriver.exe')
    print(link)
    time.sleep(3)
    driver.get(link)

    try:
        common_brands.append(driver.find_element_by_xpath(
            '//*[@id="__next"]/div/div[3]/div[2]/div/section[1]/div[2]/div/div/div[2]/div/div[1]/div/div/p[1]').text)
    except:
        common_brands.append([''])
        pass

    try:
        drug_class.append(driver.find_element_by_xpath(
            '//*[@id="__next"]/div/div[3]/div[2]/div/section[1]/div[2]/div/div/div[2]/div/div[1]/div/div/p[2]').text)
    except:
        drug_class.append([''])
        pass

    try:
        controlled_substance_classification.append(driver.find_element_by_xpath(
            '//*[@id="__next"]/div/div[3]/div[2]/div/section[1]/div[2]/div/div/div[2]/div/div[1]/div/div/p[3]').text)
    except:
        controlled_substance_classification.append([''])

        pass

    try:
        generic_status.append(driver.find_element_by_xpath(
            '//*[@id="__next"]/div/div[3]/div[2]/div/section[1]/div[2]/div/div/div[2]/div/div[2]/div[1]/div/p[1]').text)
    except:
        generic_status.append([''])

        pass

    try:
        availability.append(driver.find_element_by_xpath(
            '//*[@id="__next"]/div/div[3]/div[2]/div/section[1]/div[2]/div/div/div[2]/div/div[2]/div[1]/div/p[2]').text)
    except:
        availability.append([''])
        pass

    # writing all scrapped data into a dictionary
    drug_data = {'link2': link}
    all_variables = [availability, generic_status, controlled_substance_classification, drug_class, common_brands]

    drug_data.update({drung_info_keys[key]: (', ').join(all_variables[key]) for key in range(len(drung_info_keys))})
    # using join() to avoid parenthesis in csv/xlsx file

    return drug_data

for i in lk[110:255]:
    description = []
    name = []
    brand_name = []
    generic_name = []
    form = []
    dosage = []
    quantity = []
    # drung_info_link = []

    details_data = {'drug_id': count,'link1':i}
    driver.get(i)
    driver.execute_script('window.running_test_suite_for_foobar = false')
    time.sleep(1)
    time.sleep(1)
    print(i)
    try:
        driver.find_element_by_xpath('//*[@id="contactable-persons-modal"]/div[1]/button').click()
    finally:
        try:
            supClass = driver.find_elements_by_class_name('container-22AZo')
            des = supClass[0].find_element_by_class_name('description-2vGyn').text
            nm1 = supClass[0].find_element_by_class_name('titleStyles-i9mVW').text
            description.extend([des])
            name.append(nm1)
            time.sleep(5)
        except:
            pass
        try:
            driver.find_element_by_xpath('//*[@id="uat-dropdown-brand"]/i').click()
            generic_name.append(driver.find_element_by_xpath(
                '//*[@id="app-root"]/div/div/div/div/div/section/section/div[2]/div[2]/div[1]/div[1]/div/div[2]/ul/li[2]/button').text)
            brand_name.append(driver.find_element_by_xpath(
                '//*[@id="app-root"]/div/div/div/div/div/section/section/div[2]/div[2]/div[1]/div[1]/div/div[2]/ul/li[1]/button').text)
        except:
            pass

        try:
            driver.find_element_by_xpath('//*[@id="uat-dropdown-form"]/i').click()
            form1 = []
            for i in range(1, 4):
                form1.append(driver.find_element_by_xpath(
                    f'//*[@id="app-root"]/div/div/div/div/div/section/section/div[2]/div[2]/div[1]/div[2]/div/div[2]/ul/li[{i}]/button').text)
            form.extend(form1)
        except:
            form1 = []
            form1.append(driver.find_element_by_xpath('//*[@id="uat-dropdown-form"]/span').text)
            form.extend(form1)
            pass

        try:
            driver.find_element_by_xpath('//*[@id="uat-dropdown-dosage"]/i').click()
            # dosage1 = []
            for i in range(1, 4):
                dosage.append(driver.find_element_by_xpath(
                    f'//*[@id="app-root"]/div/div/div/div/div/section/section/div[2]/div[2]/div[1]/div[3]/div/div[2]/ul/li[{i}]/button').text)

        except:
            dosage1 = []
            dosage1.append(driver.find_element_by_xpath('//*[@id="uat-dropdown-dosage"]/span').text)
            dosage.extend(dosage1)
            pass

        try:
            driver.find_element_by_xpath('//*[@id="uat-dropdown-quantity"]/i').click()
            quantity1 = []
            for i in range(1, 4):
                quantity1.append(driver.find_element_by_xpath(
                    f'//*[@id="app-root"]/div/div/div/div/div/section/section/div[2]/div[2]/div[1]/div[4]/div/div[2]/ul/li[{i}]/button').text)
            quantity.extend(quantity1)
        except:
            quantity1 = []
            quantity1.append(driver.find_element_by_xpath('//*[@id="uat-dropdown-quantity"]/span').text)
            quantity.extend(quantity1)
            pass
        try:
            link2 = driver.find_element_by_link_text("Drug Info").get_attribute('href')
            details_data.update(get_drung_info(link2))
        except:
            print('-1')

        details_value = [description, generic_name, brand_name, form, quantity, dosage]

        details_data.update({details_keys[key]: (', ').join(details_value[key]) for key in range(len(details_value))})
        details_writer.writerow(details_data)
        count+=1
        continue

driver.close()

https://www.goodrx.com/vimpat?label_override=vimpat


C:\Users\astro\AppData\Local\Temp/ipykernel_9288/1166957422.py:78: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="contactable-persons-modal"]/div[1]/button').click()
C:\Users\astro\AppData\Local\Temp/ipykernel_9288/1166957422.py:81: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  supClass = driver.find_elements_by_class_name('container-22AZo')
C:\Users\astro\AppData\Local\Temp/ipykernel_9288/1166957422.py:90: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="uat-dropdown-brand"]/i').click()
C:\Users\astro\AppData\Local\Temp/ipykernel_9288/1166957422.py:91: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  generic_name.append(driver.find_element_by_xpath(
C:\Users\astro\AppData\Local\Temp/ipykernel_9288/1166957

https://www.goodrx.com/vimpat/what-is?label_override=vimpat


C:\Users\astro\AppData\Local\Temp/ipykernel_9288/1166957422.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  common_brands.append(driver.find_element_by_xpath(
C:\Users\astro\AppData\Local\Temp/ipykernel_9288/1166957422.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  drug_class.append(driver.find_element_by_xpath(
C:\Users\astro\AppData\Local\Temp/ipykernel_9288/1166957422.py:30: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  controlled_substance_classification.append(driver.find_element_by_xpath(
C:\Users\astro\AppData\Local\Temp/ipykernel_9288/1166957422.py:38: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  generic_status.append(driver.find_element_by_xpath(
C:\Users\astro\AppData\Local\Temp/ipykernel_9288/1166957422.py:46: DeprecationWarning: find_element_by_* command

-1
https://www.goodrx.com/lamictal?label_override=lamictal


...

https://www.goodrx.com/lamotrigine/what-is?label_override=lamictal
https://www.goodrx.com/topamax?label_override=topamax
https://www.goodrx.com/topiramate/what-is?label_override=topamax
https://www.goodrx.com/tegretol?label_override=tegretol


C:\Users\astro\AppData\Local\Temp/ipykernel_9288/1166957422.py:120: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  dosage1.append(driver.find_element_by_xpath('//*[@id="uat-dropdown-dosage"]/span').text)


https://www.goodrx.com/tegretol/what-is?label_override=tegretol
-1
https://www.goodrx.com/cialis?label_override=cialis
https://www.goodrx.com/cialis/what-is?label_override=cialis
https://www.goodrx.com/viagra?label_override=viagra
https://www.goodrx.com/viagra/what-is?label_override=viagra
https://www.goodrx.com/kybella?label_override=kybella


C:\Users\astro\AppData\Local\Temp/ipykernel_9288/1166957422.py:133: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  quantity1.append(driver.find_element_by_xpath('//*[@id="uat-dropdown-quantity"]/span').text)


https://www.goodrx.com/kybella/what-is?label_override=kybella
https://www.goodrx.com/latisse?label_override=latisse
https://www.goodrx.com/latisse/what-is?label_override=latisse
https://www.goodrx.com/hcg?label_override=hcg
https://www.goodrx.com/hcg/what-is?label_override=hcg
-1
https://www.goodrx.com/tamiflu?label_override=tamiflu
https://www.goodrx.com/tamiflu/what-is?label_override=tamiflu
https://www.goodrx.com/prevacid?label_override=prevacid
https://www.goodrx.com/lansoprazole/what-is?label_override=prevacid
https://www.goodrx.com/zantac?label_override=zantac
https://www.goodrx.com/zantac/what-is?label_override=zantac
-1
https://www.goodrx.com/carafate?label_override=carafate
https://www.goodrx.com/sucralfate/what-is?label_override=carafate
https://www.goodrx.com/decadron?label_override=decadron
https://www.goodrx.com/dexamethasone/what-is?label_override=decadron
https://www.goodrx.com/pepcid?label_override=pepcid
https://www.goodrx.com/famotidine/what-is?label_override=pepcid
h